# Add Features

In [1]:
import psycopg2 as pg
import pandas as pd
import numpy as np

from psycopg2.extras import execute_values

1. Historic wins against opponent
2. Conference avg. rank

## Example Data

In [15]:
conn = pg.connect(database='postgres',
                  user='postgres',
                  password='w207final',
                  host='35.185.225.167')

# pull regular season and tourney data to build indexes
query = '''
    SELECT *
    FROM prod.features
    WHERE "Season Type" = 'Regular'
    LIMIT 5
'''

df = pd.read_sql_query(query, conn)
conn.close()

In [16]:
df.shape

(5, 34)

In [17]:
df.head()

,Season,DayNum,Team,Opponent,Outcome,Score,OpponentScore,NumOT,WLoc,Season Type,...,WinPct,OpponentWinPct,WinPctDiff,AvgPointsFor,AvgPointsAgainst,AvgNetPointsFor,SeedDiff,OpponentAvgPointsFor,OpponentAvgPointsAgainst,OpponentAvgNetPointsFor
0,2003,68,1115,1106,0,33,43,0,A,Regular,...,None,None,None,None,None,None,None,None,None,None
1,2003,70,1290,1106,0,67,73,0,A,Regular,...,None,None,None,None,None,None,None,None,None,None
2,2003,96,1115,1106,0,33,57,0,H,Regular,...,None,None,None,None,None,None,None,None,None,None
3,2003,98,1290,1106,1,79,78,0,A,Regular,...,None,None,None,None,None,None,None,None,None,None
4,2003,110,1380,1106,1,57,54,0,H,Regular,...,None,None,None,None,None,None,None,None,None,None


## Historic Wins Against Opponent

In [64]:
conn = pg.connect(database='postgres',
                  user='postgres',
                  password='w207final',
                  host='35.185.225.167')

# pull regular season and tourney data to build indexes
query = '''
    SELECT "Season", "DayNum", "Team", "Opponent", "Outcome"
    FROM prod.features

'''
#     WHERE "Season Type" = 'Regular'



df = pd.read_sql_query(query, conn)
conn.close()

In [65]:
df.shape

(170600, 5)

In [66]:
df.head()

,Season,DayNum,Team,Opponent,Outcome
0,2003,68,1115,1106,0.0
1,2003,70,1290,1106,0.0
2,2003,96,1115,1106,0.0
3,2003,98,1290,1106,1.0
4,2003,110,1380,1106,1.0


In [67]:
# Test - filtering off team 1102 and opponent 1204, 1140

#df[['Season', 'Team' == 1102, 'Opponent' == 1204, 'Outcome']]
df[(df['Team'] == 1102) & ((df['Opponent'] == 1204) | (df['Opponent'] == 1140))]


##df[df['first_name'].notnull() & (df['nationality'] == "USA")]

,Season,DayNum,Team,Opponent,Outcome
4184,2003,91,1102,1140,0.0
4186,2003,58,1102,1204,1.0
5897,2003,34,1102,1204,1.0
7161,2004,112,1102,1140,0.0
7367,2004,82,1102,1140,1.0
7682,2003,117,1102,1140,0.0
15831,2005,112,1102,1140,1.0
34019,2007,89,1102,1140,0.0
34027,2007,120,1102,1140,0.0
42020,2008,86,1102,1140,0.0


In [68]:
# Wins vs. each opponent

# Could separate seasons, but commenting out
# wins = df[['Season', 'Team', 'Opponent', 'Outcome']].groupby(by=["Season", "Team", "Opponent"]).agg(['sum', 'count'])

# Aggregating separate seasons
wins = df[['Team', 'Opponent', 'Outcome']].groupby(by=["Team", "Opponent"]).agg(['sum', 'count'])

wins.reset_index(inplace=True)
wins.columns = [' '.join(col).strip() for col in wins.columns.values]
wins.rename(columns={'Outcome sum': 'wins_v_opp', 'Outcome count': 'games_v_opp'}, inplace=True)
wins['winpct_v_opp'] = wins['wins_v_opp'] / wins['games_v_opp']

In [69]:
wins.shape

(39618, 5)

In [70]:
wins.head()

,Team,Opponent,wins_v_opp,games_v_opp,winpct_v_opp
0,1101,1102,1.0,1,1.0
1,1101,1115,1.0,1,1.0
2,1101,1117,0.0,1,0.0
3,1101,1129,0.0,1,0.0
4,1101,1132,1.0,1,1.0


In [71]:
# Test - filtering off team 1102 and opponent 1204, 1140

wins[(wins['Team'] == 1102) & ((wins['Opponent'] == 1204) | (wins['Opponent'] == 1140))]

,Team,Opponent,wins_v_opp,games_v_opp,winpct_v_opp
57,1102,1140,4.0,19,0.210526
78,1102,1204,2.0,2,1.000000
